### TODO

- Correct it, it's finding local mimima
- Find the right way to turn the solution into integers

In [135]:
from random import randint
import sys

import numpy as np
import scipy.optimize as optimize
import pandas as pd

def init_simulation(num_nodes, 
                    num_days, 
                    icu_capacities = None, 
                    transport_capacities = None, 
                    ini_path = None, 
                    demand_min = 80, 
                    demand_max = 90,
                    icu_min = 10,
                    icu_max = 300,
                    transport_min = 20,
                    transport_max = 30):
    if ini_path:
        print("you can also upload an .ini file")
        raise NotImplemented
    else:
        demand = []
        for node in range(num_nodes):
            demand.append(list(np.random.randint(demand_min, demand_max, size = num_days)))
        
        icu_capacities = icu_capacities if icu_capacities else list(np.random.randint(icu_min, 
                                                                                      icu_max, 
                                                                                      size = num_nodes))
        transport_capacities = transport_capacities if transport_capacities else list(np.random.randint(transport_min, 
                                                                                                        transport_max, 
                                                                                                        size = num_nodes)) 
    return icu_capacities, transport_capacities, demand


def calculate_outgoing(array,day,node):
    return sum(array[day][node])


def calculate_incoming(array,day,node):
    total_outgoing = 0
    for n in range(num_nodes):
        total_outgoing = total_outgoing + array[day][n][node]
    return total_outgoing


def demand_day_node_raw(node_capacity, demand_day, left_day, received_day, demand_previous_days, left_previous_days, received_previous_days):
    return demand_day - left_day + received_day+ min(demand_previous_days-left_previous_days+received_previous_days, node_capacity)


def demand_day_node(movements,day,node):
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    total_received_patients = 0
    total_outbound_patients = 0
    accumulated_demand = 0
    for d in range(day):
        total_received_patients = total_received_patients + calculate_incoming(movements_res,d,node)
        total_outbound_patients = total_outbound_patients + calculate_outgoing(movements_res,d,node)
    accumulated_demand = sum(demands[node][:day])
    
    demand = max(0, demand_day_node_raw(node_capacities[node],demands[node][day], 
                                        calculate_outgoing(movements_res,day,node), calculate_incoming(movements_res,day,node),
                                        accumulated_demand, total_outbound_patients, total_received_patients                
                ))
    return demand


def calc_total_deaths(movements):
    total = 0
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    for node in range(num_nodes):
        for day in range(num_days):
            deaths_not_attended = max(0, demand_day_node(movements,day,node) - node_capacities[node])
            deaths_transport = prob_death_transport*movements_res[day].sum()
            total = total + deaths_not_attended + deaths_transport
    return total


def outgoing_list(movements):
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    outgoing_list = list()
    for day in range(num_days):
        for node in range(num_nodes):
            outgoing_list.append(calculate_outgoing(movements_res,day,node))
    return outgoing_list

def generate_bounds():
    outgoing_list = list()
    for day in range(num_days):
        for outgoing_node in range(num_nodes):
            for incoming_node in range(num_nodes):
                outgoing_list.append((0,transport_capacities[outgoing_node]))
    return outgoing_list

def f_cons(node_id, day):
    return lambda x: transport_capacities[node_id] - calculate_outgoing(np.reshape(x,(num_days,num_nodes,num_nodes)),day ,node_id)


def split_n(num, num_nodes):
    #TODO this function is not truly random
    pieces = []
    for idx in range(num_nodes-1):
        pieces.append(randint(1,num-sum(pieces)-num_nodes+idx))

    pieces.append(num-sum(pieces))
    return pieces

Exception reporting mode: Plain


In [104]:
prob_death_transport = 0
num_days = 10
num_nodes = 4
node_capacities, transport_capacities, demands = init_simulation(num_nodes, 
                                                                 num_days, 
                                                                 icu_capacities=[30000, 20, 80, 1000],
                                                                 transport_capacities=[50, 60, 70, 80], 
                                                                 demand_min = 60, demand_max=100)
print(f"""Randomly generated
{num_days} days, {num_nodes} hospitals
ICU capacities of hospitals: {node_capacities}
Daily transport capacities: {transport_capacities}, 
Daily simulated demand (from uniform distrution): 
{demands}""")

Randomly generated
10 days, 4 hospitals
ICU capacities of hospitals: [30000, 20, 80, 1000]
Daily transport capacities: [50, 60, 70, 80], 
Daily simulated demand (from uniform distrution): 
[[68, 78, 65, 98, 66, 78, 83, 91, 82, 67], [99, 94, 85, 77, 85, 73, 83, 66, 69, 77], [68, 71, 60, 91, 83, 81, 61, 81, 75, 78], [72, 76, 87, 64, 91, 64, 72, 88, 74, 71]]


In [244]:
#define constraints
cons = []
for node in range(num_nodes):
    for day in range(num_days):
        cons.append({'type': 'ineq', 'fun': f_cons(node,day)})

        
#no movement matrix, the base for generating random ones and one of the ones to test as init matrix 
no_movement = np.array([[[0]*num_nodes]*num_nodes]*num_days)

In [243]:
np.random.shuffle([1,0], size=num_days)

array([1, 0, 1, 0, 1, 1, 0, 0, 0, 1])

In [246]:
def random_init(knockdown_days = False, movement_intensity = 1):
    random_movement = no_movement.copy()
    #if knockdown days is active, behave like a dropdown NN layer and drop half of the days movements
    active_days = np.random.choice([1,0], size=num_days) if knockdown_days else [1]*num_days
    for day in range(num_days):
        if active_days[day]:
            for node_x in range(num_nodes):
                #splits a number in a list of numbers pseudo randomly
                split_nodes = split_n(max(num_nodes+1, 
                                          np.ceil(transport_capacities[node_x]*movement_intensity)), 
                                      num_nodes)
                for node_y in range(num_nodes):
                    if node_x != node_y: #the ones that move to "self" are actually staying
                        random_movement[day, node_x, node_y] += split_nodes[node_y]
                    
    return random_movement

In [255]:
random_init(movement_intensity=0.3, knockdown_days=True)

array([[[ 0,  1,  3,  4],
        [11,  0,  1,  2],
        [ 5,  2,  0, 11],
        [ 4,  7, 11,  0]],

       [[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[ 0,  1,  4,  7],
        [ 1,  0,  1,  2],
        [12,  5,  0,  2],
        [12,  2,  6,  0]],

       [[ 0,  9,  1,  2],
        [13,  0,  1,  2],
        [16,  2,  0,  2],
        [16,  4,  1,  0]],

       [[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[ 0,  2,  2,  3],
        [ 6,  0,  3,

### The main loop that generates random init matrices

In [286]:
%%time

#parameters to minimize
min_deaths = sys.maxsize
best_result = None
result_matrix = None
#iterate through low to high intensity movement simulation
step=0.025
steps = []
i = 0
for movement_intensity in np.arange(0, 1+step, step):
    for knockdown in (True, False):
        i += 1
        print(f"i:{i} intensity:{movement_intensity:.3f} knockdown={knockdown} ", end="")
        movement = random_init(knockdown, movement_intensity) if movement_intensity else no_movement
        movement = movement.ravel()
        bounds_movement = generate_bounds()
        #the actual
        result = optimize.minimize(calc_total_deaths, 
                                   movement, 
                                   method='SLSQP', 
                                   constraints=cons, 
                                   bounds=bounds_movement)

        current_deaths = calc_total_deaths(np.rint(result.x))
        print(f"Result:{current_deaths:.0f}", end="")
        steps.append([movement_intensity, knockdown, current_deaths])
        if min_deaths > current_deaths: 
            print(" <--best!", end="")
            min_deaths = int(current_deaths)
            best_result = result
            result_matrix = np.reshape(np.rint(result.x),(num_days,num_nodes,num_nodes))
            win_init_matrix = movement
        
        print("")
        
print("\nDone")

i:1 intensity:0.000 knockdown=True Result:189 <--best!
i:2 intensity:0.000 knockdown=False Result:189
i:3 intensity:0.025 knockdown=True Result:188 <--best!
i:4 intensity:0.025 knockdown=False Result:198
i:5 intensity:0.050 knockdown=True Result:188
i:6 intensity:0.050 knockdown=False Result:198
i:7 intensity:0.075 knockdown=True Result:191
i:8 intensity:0.075 knockdown=False Result:188
i:9 intensity:0.100 knockdown=True Result:187 <--best!
i:10 intensity:0.100 knockdown=False Result:189
i:11 intensity:0.125 knockdown=True Result:188
i:12 intensity:0.125 knockdown=False Result:190
i:13 intensity:0.150 knockdown=True Result:196
i:14 intensity:0.150 knockdown=False Result:186 <--best!
i:15 intensity:0.175 knockdown=True Result:189
i:16 intensity:0.175 knockdown=False Result:187
i:17 intensity:0.200 knockdown=True Result:186
i:18 intensity:0.200 knockdown=False Result:188
i:19 intensity:0.225 knockdown=True Result:189
i:20 intensity:0.225 knockdown=False Result:189
i:21 intensity:0.250 kn

In [281]:
df = pd.DataFrame(steps)
df.columns = ["movement_intensity", "knockdown", "result"]
df

,movement_intensity,knockdown,result
0,0.000,True,189.0
1,0.000,False,189.0
2,0.025,True,191.0
3,0.025,False,198.0
4,0.050,True,188.0
...,...,...,...
77,0.950,False,191.0
78,0.975,True,188.0
79,0.975,False,191.0
80,1.000,True,190.0


In [284]:
df.groupby("knockdown").agg([np.mean, np.median, np.std])["result"]

,mean,median,std
knockdown,,,
False,188.926829,188.0,2.961336
True,188.414634,188.0,2.559059


### These are the deaths that we get with inaction

In [264]:
calc_total_deaths(no_movement)

1457

### This is the best result found

In [265]:
min_deaths

183

### This the movement matrix

In [266]:
result_matrix

array([[[ 0.,  0.,  0.,  0.],
        [39.,  0.,  2., 20.],
        [45.,  0.,  0., 25.],
        [19.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [39.,  0.,  2., 20.],
        [45.,  0.,  0., 25.],
        [19.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [39.,  0.,  2., 20.],
        [45.,  0.,  0., 25.],
        [19.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [38.,  0.,  2., 19.],
        [43.,  0.,  0., 25.],
        [28.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [39.,  0.,  2., 20.],
        [45.,  0.,  0., 25.],
        [19.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [39.,  0.,  2., 20.],
        [45.,  0.,  0., 25.],
        [19.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  2.],
        [41.,  0.,  0., 19.],
        [46.,  0.,  0., 24.],
        [25.,  0.,  2.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [41.,  0.,  1., 19.],
        [42.,  0.,  0., 24.],
        [23.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., 

In [267]:
for node in range(num_nodes):
    for day in range(num_days):
        print(f'Node: {node}, Day: {day}, Demand: {demand_day_node(np.rint(result.x),day,node)}')

Node: 0, Day: 0, Demand: 196.0
Node: 0, Day: 1, Demand: 373.0
Node: 0, Day: 2, Demand: 612.0
Node: 0, Day: 3, Demand: 855.0
Node: 0, Day: 4, Demand: 1107.0
Node: 0, Day: 5, Demand: 1304.0
Node: 0, Day: 6, Demand: 1547.0
Node: 0, Day: 7, Demand: 1676.0
Node: 0, Day: 8, Demand: 1855.0
Node: 0, Day: 9, Demand: 2014.0
Node: 1, Day: 0, Demand: 40.0
Node: 1, Day: 1, Demand: 53.0
Node: 1, Day: 2, Demand: 45.0
Node: 1, Day: 3, Demand: 37.0
Node: 1, Day: 4, Demand: 46.0
Node: 1, Day: 5, Demand: 32.0
Node: 1, Day: 6, Demand: 42.0
Node: 1, Day: 7, Demand: 25.0
Node: 1, Day: 8, Demand: 29.0
Node: 1, Day: 9, Demand: 36.0
Node: 2, Day: 0, Demand: 0
Node: 2, Day: 1, Demand: 6.0
Node: 2, Day: 2, Demand: 0
Node: 2, Day: 3, Demand: 20.0
Node: 2, Day: 4, Demand: 33.0
Node: 2, Day: 5, Demand: 45.0
Node: 2, Day: 6, Demand: 38.0
Node: 2, Day: 7, Demand: 57.0
Node: 2, Day: 8, Demand: 62.0
Node: 2, Day: 9, Demand: 73.0
Node: 3, Day: 0, Demand: 73.0
Node: 3, Day: 1, Demand: 174.0
Node: 3, Day: 2, Demand: 215.0